## Juntando o Histórico Ampliado (JSON)

O histórico estendido de streaming do Spotify é geralmente armazenado em arquivos JSON separados, especialmente se você estiver exportando dados de várias contas ou períodos diferentes. A etapa a seguir explica como consolidar esses arquivos JSON em um único DataFrame para análise posterior.

Este processo consolida com sucesso todos os arquivos JSON do histórico estendido em um único DataFrame, tornando-o pronto para análises adicionais, como buscar informações e imagens de músicas, álbuns e artistas.

In [12]:
import os
import json
import pandas as pd

# Define relative paths
PATH_EXTENDED_HISTORY = 'Spotify Data/raw/StreamingHistory_Extended/'
PATH_OUT = 'Spotify Data/Processed/' 

# Get a list of all JSON files in the directory
json_files = [pos_json for pos_json in os.listdir(PATH_EXTENDED_HISTORY ) if pos_json.endswith('.json')]

# Initialize an empty list to hold DataFrames
dfs = []

# Load the data from each JSON file and append it to the DataFrame list
for index, js in enumerate(json_files):
    with open(os.path.join(PATH_EXTENDED_HISTORY , js)) as json_file:
        json_text = json.load(json_file)
        temp_df = pd.json_normalize(json_text)
        dfs.append(temp_df)

# Concatenate all the DataFrames in the list into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

df.drop(['platform','username', 'conn_country' ,'ip_addr_decrypted', 'user_agent_decrypted'], axis=1, inplace=True)

# Cast object columns containing only 'True' and 'False' strings to bool dtype
for col in df.columns:
    if df[col].dtype == 'object' and all(df[col].dropna().apply(lambda x: x in [True, False, 'True', 'False'])):
        df[col] = df[col].astype(bool)

display(df.head(5))

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2014-09-29T17:53:45Z,8135,I Am the Man,Freshlyground,Nomvula,spotify:track:019m7qUQerzKMWNlS7sO2o,None,None,None,clickrow,clickrow,False,True,False,0.0,False
1,2014-09-29T17:59:02Z,317106,Vanish,Freshlyground,Nomvula,spotify:track:5ruzfgWfhgusrn772s9dC5,None,None,None,clickrow,trackdone,False,False,False,0.0,False
2,2014-09-29T18:54:09Z,28932,Zithande,Freshlyground,Nomvula,spotify:track:6POnCFfPhnIAL5VSh4YqyC,None,None,None,trackdone,endplay,False,True,False,0.0,False
3,2014-10-07T00:46:31Z,324000,My Little Brown Book,Duke Ellington,Duke Ellington & John Coltrane,spotify:track:4TWObpR5vN8WRzs2FdRRyo,None,None,None,clickrow,trackdone,False,False,False,0.0,False
4,2014-10-07T00:52:22Z,350426,Circle,Miles Davis,The Complete Columbia Studio Recordings Of The...,spotify:track:5Vk50D8fYV6vFGznPKSi6K,None,None,None,trackdone,trackdone,False,False,False,0.0,False



## Processando e Organizando as Curtidas (💚)

Neste segmento de código, estamos processando um arquivo JSON que contém informações sobre as faixas curtidas ("likes") pelo usuário no Spotify. Esses dados são extraídos e organizados em um DataFrame para análise posterior. Vamos explorar cada etapa em detalhes:

1. **Definir o Caminho do Arquivo**: O caminho para o arquivo JSON que contém as informações das faixas curtidas é definido.

2. **Ler o Arquivo JSON**: O arquivo é lido e os dados são carregados como um objeto JSON.

3. **Extrair Dados de Faixas**: A chave 'tracks' é extraída do objeto JSON, que contém todas as informações sobre as faixas curtidas.

4. **Converter em DataFrame**: Os dados das faixas são convertidos em um DataFrame do Pandas para facilitar a manipulação.

5. **Extrair Colunas Relevantes**: Apenas as colunas relevantes são mantidas, incluindo o artista, álbum, faixa e URI (Identificador Uniforme de Recursos).

6. **Dividir a Coluna URI**: A coluna 'uri' é dividida usando o delimitador ':', que resulta em novas colunas contendo a fonte, o tipo e o ID da faixa.

7. **Remover a Coluna URI Original**: A coluna 'uri' original é removida, já que suas partes foram extraídas.

8. **Exibir o DataFrame Final**: O DataFrame final é exibido, contendo as informações organizadas das faixas curtidas.

Esse processo organiza as informações de curtidas em uma forma tabular, tornando-as prontas para análise adicional, visualização ou integração com outros conjuntos de dados. É uma etapa essencial para entender as preferências musicais do usuário e pode ser usada para recomendações personalizadas, análise de tendências e muito mais.

In [2]:
import pandas as pd
import json
import os

# Define the file path
PATH_SIMPLE_HISTORY = 'Spotify Data/raw/MyDataSimple/' 
file_path = os.path.join(PATH_SIMPLE_HISTORY, 'YourLibrary.json')

# Read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract the 'tracks' key from the JSON object
tracks_data = data['tracks']

# Convert the tracks_data object to a DataFrame
df_Curtidas = pd.DataFrame(tracks_data)

# Extract the required columns
df_Curtidas = df_Curtidas[['artist', 'album', 'track', 'uri']]

# Split the 'uri' column by the delimiter ':'
df_Curtidas[['source', 'type', 'id']] = df_Curtidas['uri'].str.split(':', expand=True)

# Drop the original 'uri' column
df_Curtidas.drop('uri', axis=1, inplace=True)

# Display the final DataFrame
display(df_Curtidas)

,artist,album,track,source,type,id
0,Usher,Confessions (Expanded Edition),Seduction,spotify,track,2qVqxcTXGugY41u8Sr8xfp
1,Academia da Berlinda,Olindance,Praia do L,spotify,track,5Z9MaG5neshsgau3TTvlUR
2,The Weeknd,Starboy,I Feel It Coming,spotify,track,4RepvCWqsP6zBuzvwYibAS
3,Kygo,Cloud Nine,Serious (feat. Matt Corby),spotify,track,18YIDJGWXocUPFbAzTaq8g
4,Gyvus,Blue Sky,Blue Sky,spotify,track,3cdXOVNgZOXSQSvOl2F9W3
...,...,...,...,...,...,...
1082,Bakar,Hell N Back,Hell N Back,spotify,track,3NRql0A1Ef4RCvT473iqgD
1083,Cidade Negra,Hits,Sábado a Noite (feat. Lulu Santos),spotify,track,1m66FY5cvtDcxuVvsZ3tER
1084,Rael,Flor de Aruanda,Flor de Aruanda,spotify,track,53UWKafwpKToHDBB286Q0e
1085,Barão Vermelho,Maior Abandonado,Maior Abandonado,spotify,track,30VpfIcLiuLYN3HHjrpWJ7


## Processando e Organizando as Playlists

Neste segmento de código, estamos processando um arquivo JSON que contém informações sobre as playlists criadas pelo usuário no Spotify. Esses dados são extraídos e organizados em um DataFrame para análise posterior. Vamos explorar cada etapa em detalhes:

1. **Definir o Caminho do Arquivo**: O caminho para o arquivo JSON que contém as informações das playlists é definido.

2. **Ler o Arquivo JSON**: O arquivo é lido e os dados são carregados como um objeto JSON.

3. **Extrair Dados de Playlists**: A chave 'playlists' é extraída do objeto JSON, que contém todas as informações sobre as playlists criadas pelo usuário.

4. **Converter em DataFrame**: Os dados das playlists são convertidos em um DataFrame do Pandas para facilitar a manipulação.

5. **Extrair Colunas Relevantes**: Apenas as colunas relevantes são mantidas, incluindo o nome da playlist, data da última modificação, itens (faixas), descrição e número de seguidores.

6. **Exibir o DataFrame Final**: O DataFrame final é exibido, contendo as informações organizadas das playlists.

Esse processo organiza as informações das playlists em uma forma tabular, tornando-as prontas para análise adicional, visualização ou integração com outros conjuntos de dados. É uma etapa essencial para entender a estrutura e o conteúdo das playlists do usuário, fornecendo insights valiosos sobre as preferências musicais, o engajamento com diferentes artistas e gêneros, e a dinâmica de compartilhamento e seguimento de playlists.

In [3]:
file_path = os.path.join(PATH_SIMPLE_HISTORY, 'Playlist1.json')

# Read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Extract the 'playlists' key from the JSON object
playlists_data = data['playlists']

# Convert the playlists_data object to a DataFrame
df_playlists = pd.DataFrame(playlists_data)

# Extract the required columns
df_playlists = df_playlists[['name', 'lastModifiedDate', 'items', 'description', 'numberOfFollowers']]

# Display the final DataFrame
display(df_playlists)


,name,lastModifiedDate,items,description,numberOfFollowers
0,CHILL FUNKSTEP SOUL,2023-07-13,[{'track': {'trackName': 'Homie. Lover. Friend...,None,1
1,Hits from 80's,2023-07-14,"[{'track': {'trackName': 'September', 'artistN...",Made with playlistai.app,0
2,Mechanical Podcast,2023-03-13,"[{'track': None, 'episode': {'episodeName': 'E...",None,0
3,Drive To The Moon,2023-07-20,"[{'track': {'trackName': 'Ghostly', 'artistNam...",None,4
4,Hora de Brincar,2023-07-05,"[{'track': {'trackName': 'Lá no Galinheiro', '...",None,2
5,Rock Is Dead,2023-03-13,[{'track': {'trackName': 'Killing In The Name'...,None,0
6,Data Podcasts,2023-03-13,"[{'track': None, 'episode': {'episodeName': '0...",None,0
7,Do Surf,2023-03-12,"[{'track': {'trackName': 'Santeria', 'artistNa...",None,11
8,Go Easy,2023-05-01,[{'track': {'trackName': 'Queen of California ...,"Chill guitar, peacefull feeling",4
9,Chill Hill,2023-06-16,"[{'track': {'trackName': 'Departure', 'artistN...",Sit back and relax,26


## Extração e Organização das Músicas e Episódios das Playlists

Neste segmento do código, estamos aprofundando as playlists do usuário para extrair informações sobre as músicas e episódios que elas contêm. A estrutura de dados envolve objetos aninhados, então o processo envolve várias etapas de manipulação para transformar os dados em uma forma tabular. Aqui está o que acontece em cada etapa:

1. **Selecionar Colunas Relevantes**: As colunas 'name' (nome da playlist) e 'items' (faixas/episódios na playlist) são selecionadas.

2. **Expandir a Coluna 'Items'**: A coluna 'items' contém uma lista de objetos para cada faixa ou episódio. Esses objetos são extraídos e organizados em uma lista de dicionários, incluindo o nome da playlist e os detalhes da faixa/episódio.

3. **Criar DataFrame para Músicas e Episódios**: Um novo DataFrame é criado com os dados das músicas e episódios, incluindo colunas separadas para 'track' e 'episode'.

4. **Expandir as Colunas 'Track' e 'Episode'**: As colunas 'track' e 'episode' são expandidas em colunas separadas para nome da faixa, nome do artista, nome do álbum, URI da faixa, nome do episódio, nome do show e URI do episódio.

5. **Combinar e Reorganizar Colunas**: Várias colunas são combinadas e reorganizadas para criar um formato coeso. Isso inclui a combinação de URIs de faixas e episódios, nomes de faixas e episódios, e nomes de artistas e shows.

6. **Extrair ID**: O ID da faixa ou episódio é extraído da URI.

7. **Selecionar Colunas Finais**: As colunas finais são selecionadas para criar o DataFrame final, incluindo o nome da playlist, nome do álbum, nome da faixa/episódio, nome do criador (artista/show), URI e ID.

8. **Exibir o DataFrame Final**: O DataFrame final é exibido, contendo as informações organizadas das músicas e episódios nas playlists do usuário.

Este processo cria uma visão detalhada e estruturada das músicas e episódios nas playlists do usuário. Ele revela informações valiosas sobre as preferências do usuário, o conteúdo das playlists e a relação entre músicas, artistas, álbuns e episódios de podcast. Isso pode ser usado para análise de tendências, recomendação personalizada e insights sobre o comportamento do usuário no Spotify.

In [4]:
# Select the required columns
df_items = df_playlists[['name', 'items']].copy()

# Expand the 'items' column
items_data = []
for index, row in df_items.iterrows():
    for item in row['items']:
        items_data.append({
            'name': row['name'],
            'track': item.get('track'),
            'episode': item.get('episode'),
        })

df_playlist_songs = pd.DataFrame(items_data)

# Expand the 'track' and 'episode' columns
df_playlist_songs['trackName'] = df_playlist_songs['track'].apply(lambda x: x.get('trackName') if x else '')
df_playlist_songs['artistName'] = df_playlist_songs['track'].apply(lambda x: x.get('artistName') if x else '')
df_playlist_songs['albumName'] = df_playlist_songs['track'].apply(lambda x: x.get('albumName') if x else '')
df_playlist_songs['trackUri'] = df_playlist_songs['track'].apply(lambda x: x.get('trackUri') if x else '')
df_playlist_songs['episodeName'] = df_playlist_songs['episode'].apply(lambda x: x.get('episodeName') if x else '')
df_playlist_songs['showName'] = df_playlist_songs['episode'].apply(lambda x: x.get('showName') if x else '')
df_playlist_songs['episodeUri'] = df_playlist_songs['episode'].apply(lambda x: x.get('episodeUri') if x else '')

# Drop the original 'track' and 'episode' columns
df_playlist_songs.drop(['track', 'episode'], axis=1, inplace=True)

# Combine 'trackUri' and 'episodeUri' into a new 'Uri' column
df_playlist_songs['Uri'] = df_playlist_songs['trackUri'] + df_playlist_songs['episodeUri']

# Combine 'trackName' and 'episodeName' into a new 'track' column
df_playlist_songs['track'] = df_playlist_songs['trackName'] + df_playlist_songs['episodeName']

# Combine 'artistName' and 'showName' into a new 'Creator' column
df_playlist_songs['Creator'] = df_playlist_songs['artistName'] + df_playlist_songs['showName']

# Extract the text after the delimiter ':' from the 'Uri' column
df_playlist_songs['id'] = df_playlist_songs['Uri'].apply(lambda x: x.split(':')[2] if ':' in x else x)

# ... (Same code as above)

# Select only the desired columns
df_playlist_songs = df_playlist_songs[['name', 'albumName', 'track', 'Creator', 'Uri', 'id']]

# Display the final DataFrame
display(df_playlist_songs)


,name,albumName,track,Creator,Uri,id
0,CHILL FUNKSTEP SOUL,For Real. (The Remixes),Homie. Lover. Friend. - Moods Remix,Secret Rendezvous,spotify:track:3czDreyMrjxqTKLFBxe4Ij,3czDreyMrjxqTKLFBxe4Ij
1,CHILL FUNKSTEP SOUL,I Had a Dream About You,I Had a Dream About You,Bungalow,spotify:track:66XQI03EPMRiFHtqmdVluM,66XQI03EPMRiFHtqmdVluM
2,CHILL FUNKSTEP SOUL,Collective Soul,Where The River Flows,Collective Soul,spotify:track:5c3K5Lwh6vcHi98qNYSNx8,5c3K5Lwh6vcHi98qNYSNx8
3,CHILL FUNKSTEP SOUL,Chillhop Essentials Spring 2018,Handsome People,Birocratic,spotify:track:62I8hspNevExL88QkIRp7Q,62I8hspNevExL88QkIRp7Q
4,CHILL FUNKSTEP SOUL,Coastal Era,Coastal Era,Billa Qause,spotify:track:3UaU98H1BUAXKSSI5lCjwB,3UaU98H1BUAXKSSI5lCjwB
...,...,...,...,...,...,...
1478,Músicas Brasileiras Variadas,Pontos de Exclamação,Pontos de Exclamação,Jovem Dionisio,spotify:track:2mlsK27VNUj8y08BvtIbXu,2mlsK27VNUj8y08BvtIbXu
1479,Músicas Brasileiras Variadas,Umas E Outras,Jorge Maravilha,Chico Buarque,spotify:track:6K022fAYR1Q2BQwKfugUi3,6K022fAYR1Q2BQwKfugUi3
1480,Músicas Brasileiras Variadas,Di Melo,A Vida Em Seus Metodos Diz Calma,Di Melo,spotify:track:6niY7onsGQgK2sgF9XrYir,6niY7onsGQgK2sgF9XrYir
1481,Músicas Brasileiras Variadas,Di Melo,Kilario,Di Melo,spotify:track:00Xcf47ApZUt6omH2RgdUM,00Xcf47ApZUt6omH2RgdUM


## Consolidação e Tratamento de Dados de Faixas e Episódios (Histórico Ampliado + API)

Neste segmento do código, estamos consolidando e tratando os dados relacionados às faixas e episódios. Este é um passo crucial para preparar os dados para análise, e envolve várias etapas de limpeza, transformação e fusão de dados. Aqui está o que acontece em cada etapa:

1. **Carregar Detalhes dos Episódios**: Os detalhes dos episódios são carregados de um arquivo CSV, filtrados para remover linhas inválidas e processados para ajustar os tipos de dados e nomes de colunas.

2. **Carregar Detalhes dos Álbuns e Artistas**: Os detalhes dos álbuns e artistas são carregados de arquivos CSV separados.

3. **Carregar e Fundir Detalhes das Faixas**: Os detalhes das faixas são carregados de um arquivo CSV e então fundidos com os detalhes dos artistas e álbuns. Isso cria um DataFrame abrangente contendo informações sobre as faixas, álbuns e artistas.

4. **Converter Tipos de Dados**: Várias colunas são convertidas para os tipos de dados desejados, incluindo strings, inteiros e datas.

5. **Concatenar Detalhes de Faixas e Episódios**: Os detalhes das faixas e episódios são concatenados em um único DataFrame, garantindo que todas as informações relevantes estejam em um único lugar.

6. **Tratar Valores Vazios e Nulos**: Valores vazios e nulos são tratados em várias colunas, incluindo 'genres' e 'artist_image'. Isso inclui a substituição de valores vazios por um valor padrão (por exemplo, 'não identificado') e o preenchimento de valores nulos com valores correspondentes de outras colunas.

7. **Filtrar Linhas Inválidas**: Linhas onde o nome da faixa/episódio é nulo ou vazio são filtradas para garantir que apenas dados válidos sejam incluídos.

8. **Substituir Valores em 'artist_image'**: Valores vazios na coluna 'artist_image' são substituídos pelos valores correspondentes na coluna 'album_image_url'.

9. **Exibir o DataFrame Final**: O DataFrame final consolidado é exibido, contendo informações detalhadas e tratadas sobre faixas, episódios, álbuns e artistas.

Este processo resulta em um conjunto de dados limpo e bem estruturado que está pronto para análise. Ele inclui informações ricas sobre faixas, episódios, artistas e álbuns, e permite uma análise profunda dos hábitos de escuta, tendências, preferências e muito mais. A consolidação e o tratamento adequados dos dados são fundamentais para garantir que as análises subsequentes sejam precisas e significativas.

In [9]:
import pandas as pd

# Define the file path
file_path = os.path.join(PATH_OUT, 'EpisodesDetails.csv')

# Read the CSV file into df_Episodes
df_Episodes = pd.read_csv(file_path)

# Filter rows where 'id' is not an empty string
df_Episodes = df_Episodes[df_Episodes['id'] != ""]

# Change the data type of the 'release_date' column to datetime
df_Episodes['release_date'] = pd.to_datetime(df_Episodes['release_date'])

# Rename the 'genre' column to 'genres'
df_Episodes.rename(columns={'genre': 'genres'}, inplace=True)

# Define the file path for Album
file_path_album = os.path.join(PATH_OUT, 'AlbumDetails.csv')

# Read the CSV file into df_album
df_album = pd.read_csv(file_path_album, encoding='utf-8')

# Define the file path for Artist
file_path_artist = os.path.join(PATH_OUT, 'ArtistDetail.csv')

# Read the CSV file into df_artist
df_artist = pd.read_csv(file_path_artist, encoding='utf-8')

# Read the CSV file TrackDetails.csv
file_path_track_details = os.path.join(PATH_OUT, 'TrackDetails.csv')
df_track = pd.read_csv(file_path_track_details, encoding='utf-8')

# Perform a left outer join with Artist on artist_id
df_track = df_track.merge(df_artist, left_on='artist_id', right_on='id', suffixes=('', '_artist'), how='left')

# Rename the columns artist_genres and artist_image
df_track.rename(columns={'artist_genres': 'artist_genres', 'artist_image': 'artist_image'}, inplace=True)

# Perform a left outer join with Album on album_id
df_track = df_track.merge(df_album, left_on='album_id', right_on='id', suffixes=('', '_album'), how='left')

# Rename the columns genres and label from Album
df_track.rename(columns={'genres': 'genres', 'label': 'label'}, inplace=True)

# Convert specific columns to the desired data types
df_track = df_track.astype({
    'genres': 'str',
    'id': 'str',
    'name': 'str',
    'uri': 'str',
    'album_image_url': 'str',
    'release_date': 'datetime64',
    'popularity': 'int64',
    'artist': 'str',
    'artist_id': 'str',
    'album': 'str',
    'album_id': 'str',
    'artist_genres': 'str',
    'artist_image': 'str',
    'label': 'str'
})

# Concatenate with EpisodesDetails
TrackEpisodes_df = pd.concat([df_track, df_Episodes], ignore_index=True)

# Replace empty strings and null values
TrackEpisodes_df['genres'].replace({'': 'não identificado'}, inplace=True)
TrackEpisodes_df['genres'].fillna('não identificado', inplace=True)
TrackEpisodes_df['artist_image'].fillna(TrackEpisodes_df['album_image_url'], inplace=True)
TrackEpisodes_df['album'].fillna(TrackEpisodes_df['artist'], inplace=True)

# Filter rows where name is not null and not empty
TrackEpisodes_df = TrackEpisodes_df[TrackEpisodes_df['name'].notnull() & (TrackEpisodes_df['name'] != '')]

# Replace empty strings in artist_image with the corresponding values in album_image_url
TrackEpisodes_df['artist_image'] = TrackEpisodes_df.apply(
    lambda row: row['album_image_url'] if row['artist_image'] == '' else row['artist_image'],
    axis=1
)

# TrackEpisodes_df now contains the processed data
display(TrackEpisodes_df.head(4))

,id,name,uri,album_image_url,release_date,popularity,artist,artist_id,album,album_id,id_artist,name_artist,artist_genres,artist_image,id_album,name_album,uri_album,genres,label
0,019m7qUQerzKMWNlS7sO2o,I Am the Man,spotify:track:019m7qUQerzKMWNlS7sO2o,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,15.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,7AcV1lk8Zrgo1691PDWEle,Freshlyground,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,2kKXMbWFVcqYms0Y5WsjFx,Nomvula,spotify:album:2kKXMbWFVcqYms0Y5WsjFx,nan,Freeground Records J
1,5ruzfgWfhgusrn772s9dC5,Vanish,spotify:track:5ruzfgWfhgusrn772s9dC5,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,13.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,7AcV1lk8Zrgo1691PDWEle,Freshlyground,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,2kKXMbWFVcqYms0Y5WsjFx,Nomvula,spotify:album:2kKXMbWFVcqYms0Y5WsjFx,nan,Freeground Records J
2,6POnCFfPhnIAL5VSh4YqyC,Zithande,spotify:track:6POnCFfPhnIAL5VSh4YqyC,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,24.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,7AcV1lk8Zrgo1691PDWEle,Freshlyground,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,2kKXMbWFVcqYms0Y5WsjFx,Nomvula,spotify:album:2kKXMbWFVcqYms0Y5WsjFx,nan,Freeground Records J
3,4TWObpR5vN8WRzs2FdRRyo,My Little Brown Book,spotify:track:4TWObpR5vN8WRzs2FdRRyo,https://i.scdn.co/image/ab67616d0000b27392257f...,1963-02-01,0.0,Duke Ellington,4F7Q5NV6h5TSwCainz8S5A,Duke Ellington & John Coltrane,7qsnx3thQiQabvDkHfqI1m,4F7Q5NV6h5TSwCainz8S5A,Duke Ellington,african-american classical,https://i.scdn.co/image/1e24691a352233bbe989a3...,7qsnx3thQiQabvDkHfqI1m,Duke Ellington & John Coltrane,spotify:album:7qsnx3thQiQabvDkHfqI1m,nan,Impulse!


## Criação da Tabela Dimensão para o Power BI

Neste segmento do código, estamos construindo a tabela dimensão, que servirá como um complemento à tabela fato, fornecendo informações detalhadas sobre músicas, artistas, álbuns e outros metadados relevantes. A tabela dimensão é crucial para enriquecer a análise, permitindo uma compreensão mais profunda das características das músicas e episódios escutados.

### Etapas

1. **Mesclar Dados**: A tabela dimensão é criada mesclando dados de múltiplas fontes, incluindo detalhes de faixas, episódios, álbuns, artistas e informações de curtidas e playlists.

2. **Adicionar Coluna de Playlist**: Uma coluna 'Playlist' é adicionada para classificar as músicas em diferentes categorias, como "Curtida" ou "Não é Sua Playlist", com base em condições específicas.

3. **Renomear Colunas**: Algumas colunas são renomeadas para nomes mais descritivos, facilitando a interpretação.

4. **Tratar a Coluna de Gênero**: A coluna 'genre' é tratada, substituindo valores inválidos e preenchendo valores nulos com informações relevantes de outros campos.

5. **Filtrar Linhas Vazias**: Linhas contendo valores em branco em colunas específicas são removidas para manter a consistência dos dados.

6. **Preencher e Combinar Colunas**: Valores nulos em algumas colunas são preenchidos, e informações específicas são combinadas para formar novas colunas, como 'Pesquisa', que agrega detalhes de faixa, artista, álbum e gênero.

7. **Selecionar Colunas Relevantes**: Somente as colunas necessárias para análise são selecionadas, formando o DataFrame final.

8. **Salvar em Arquivo CSV**: O DataFrame final é salvo em um arquivo CSV, pronto para ser importado no Power BI.

A tabela dimensão criada aqui complementa a tabela fato, fornecendo informações ricas e detalhadas sobre as músicas e episódios ouvidos. Esses dados enriquecidos permitem uma análise mais complexa e visualizações mais atraentes no Power BI, contribuindo para uma compreensão mais profunda dos hábitos e preferências musicais do usuário.

In [10]:
import numpy as np

df_Curtidas['Curtida']  = 'Curtida'

# Merge TrackEpisodes_df and Curtidas
merged_df = TrackEpisodes_df.merge(df_Curtidas[['id', 'Curtida']], on='id', how='left', suffixes=('', '_Curtidas'))

# Left join with Playlists_songs on the "id" column
merged_df = merged_df.merge(df_playlist_songs[['id', 'name']], on='id', how='left', suffixes=('', '_Playlist')).drop_duplicates(subset=['id'])

# Add Playlist column based on conditions
merged_df['Playlist'] = merged_df.apply(lambda row: row['name_Playlist'] if pd.notnull(row['name_Playlist']) else ('Curtida' if pd.notnull(row['Curtida']) else 'Não é Sua Playlist'), axis=1)

# Rename columns
merged_df.rename(columns={
    'name': 'trackName',
    'uri': 'Uri',
    'artist': 'artistName',
    'name_album': 'album_name',
    'genres': 'genre'
}, inplace=True)

# Replace the string "nan" with actual NaN values in the 'genre' column
merged_df['genre'].replace('nan', np.nan, inplace=True)

# Fill NaN values in 'genre' with values from 'artist_genres'
merged_df['genre'] = merged_df['genre'].fillna(merged_df['artist_genres'])

# Filter rows that have all blank values in certain columns
merged_df = merged_df[~merged_df[['trackName', 'artistName', 'album_name', 'genre']].apply(lambda x: all(x.isin([None, ''])), axis=1)]

# Replace 'album_name' with 'artistName' if 'album_name' is blank and 'Uri' contains 'episode'
merged_df['album_name'].fillna(merged_df['artistName'], inplace=True)
merged_df['album_name'] = merged_df.apply(lambda row: row['artistName'] if ('episode' in row['Uri']) else row['album_name'], axis=1)

# Combine columns for search
merged_df['Pesquisa'] = merged_df[['trackName', 'artistName', 'album_name', 'genre']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

# Select only the required columns
dSongInfo = merged_df[['id', 'trackName', 'Uri', 'album_image_url', 'release_date', 'popularity', 'artistName', 'artist_id', 'album_name', 'album_id', 'artist_genres', 'artist_image', 'genre', 'label', 'Curtida', 'Playlist', 'Pesquisa']]

# Display the final DataFrame
display(dSongInfo.head())

# Save to CSV file
save_file_artist = os.path.join(PATH_OUT, f"dSongsInfo.csv")
dSongInfo.to_csv(save_file_artist, index=False)

,id,trackName,Uri,album_image_url,release_date,popularity,artistName,artist_id,album_name,album_id,artist_genres,artist_image,genre,label,Curtida,Playlist,Pesquisa
0,019m7qUQerzKMWNlS7sO2o,I Am the Man,spotify:track:019m7qUQerzKMWNlS7sO2o,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,15.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,african rock,Freeground Records J,NaN,Não é Sua Playlist,I Am the Man Freshlyground Nomvula african rock
1,5ruzfgWfhgusrn772s9dC5,Vanish,spotify:track:5ruzfgWfhgusrn772s9dC5,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,13.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,african rock,Freeground Records J,NaN,Não é Sua Playlist,Vanish Freshlyground Nomvula african rock
2,6POnCFfPhnIAL5VSh4YqyC,Zithande,spotify:track:6POnCFfPhnIAL5VSh4YqyC,https://i.scdn.co/image/ab67616d0000b27330ce2a...,2004-09-03,24.0,Freshlyground,7AcV1lk8Zrgo1691PDWEle,Nomvula,2kKXMbWFVcqYms0Y5WsjFx,african rock,https://i.scdn.co/image/cdc8cf94774db4f0066ca1...,african rock,Freeground Records J,NaN,Não é Sua Playlist,Zithande Freshlyground Nomvula african rock
3,4TWObpR5vN8WRzs2FdRRyo,My Little Brown Book,spotify:track:4TWObpR5vN8WRzs2FdRRyo,https://i.scdn.co/image/ab67616d0000b27392257f...,1963-02-01,0.0,Duke Ellington,4F7Q5NV6h5TSwCainz8S5A,Duke Ellington & John Coltrane,7qsnx3thQiQabvDkHfqI1m,african-american classical,https://i.scdn.co/image/1e24691a352233bbe989a3...,african-american classical,Impulse!,NaN,Não é Sua Playlist,My Little Brown Book Duke Ellington Duke Ellin...
4,5Vk50D8fYV6vFGznPKSi6K,Circle,spotify:track:5Vk50D8fYV6vFGznPKSi6K,https://i.scdn.co/image/ab67616d0000b2734f3b14...,1998-03-17,10.0,Miles Davis,0kbYTNQb4Pb1rPbbaF0pT4,The Complete Columbia Studio Recordings Of The...,1mM5BwiiPqVq8T0vucuKhI,cool jazz,https://i.scdn.co/image/423e826b3c1b23930a255d...,cool jazz,Columbia/Legacy,NaN,Não é Sua Playlist,Circle Miles Davis The Complete Columbia Studi...


## Criação da Tabela Fato para o Power BI

Neste segmento do código, estamos construindo a tabela fato, que servirá como a principal fonte de dados para as análises no Power BI. A tabela fato contém informações essenciais sobre o histórico de streaming do Spotify, como data, horário, música tocada, artista, álbum, duração e outros detalhes relevantes.

### Etapas

1. **Combinar Colunas**: As colunas relativas ao nome da música, nome do álbum e URI são combinadas para consolidar informações.

2. **Transformar Tipos de Colunas**: Algumas colunas, como 'shuffle', 'data' e 'ms_played', são transformadas em tipos de dados adequados (booleano, data, tempo e inteiro).

3. **Inserir Coluna de Hora**: Uma nova coluna 'Hora' é criada, representando a hora do dia.

4. **Filtrar Linhas**: Linhas com valores vazios na coluna 'URI' são filtradas.

5. **Inserir Colunas 'Dia da Semana' e 'Período'**: Novas colunas são inseridas para representar o dia da semana e o período do dia (Madrugada, Manhã, Tarde, Noite).

6. **Renomear Colunas**: As colunas são renomeadas para nomes mais descritivos e legíveis.

7. **Transformar Coluna 'Artista'**: A coluna 'Artista' é transformada em tipo de texto.

8. **Inserir Coluna 'id'**: A coluna 'id' é inserida, extraindo a parte relevante da coluna 'URI'.

9. **Selecionar Colunas Relevantes**: Apenas as colunas necessárias para análise são selecionadas para formar o DataFrame final.

10. **Salvar em Arquivo CSV**: O DataFrame é salvo em um arquivo CSV para uso no Power BI.

A tabela fato criada aqui contém todas as informações essenciais necessárias para realizar uma análise profunda do histórico de streaming do Spotify. Essa tabela será a espinha dorsal das visualizações e análises no Power BI, permitindo insights sobre padrões de escuta, preferências musicais e comportamento do usuário ao longo do tempo.

In [11]:
import pandas as pd
import numpy as np

# Combine columns
fHistory = df.copy()
fHistory['Name'] = fHistory['master_metadata_track_name'].fillna('') + fHistory['episode_name'].fillna('')
fHistory['Album'] = fHistory['episode_show_name'].fillna('') + fHistory['master_metadata_album_album_name'].fillna('')
fHistory['URI'] = fHistory['spotify_track_uri'].fillna('') + fHistory['spotify_episode_uri'].fillna('')

# Transform "shuffle" column to logical type
fHistory['shuffle'] = fHistory['shuffle'].astype(bool)

# Split "ts" column by delimiter "T" into "data" and "timestamp"
fHistory[['data', 'timestamp']] = fHistory['ts'].str.split('T', expand=True)

# Transform column types
fHistory['data'] = pd.to_datetime(fHistory['data']).dt.date
fHistory['timestamp'] = pd.to_datetime(fHistory['timestamp']).dt.time
fHistory['ms_played'] = fHistory['ms_played'].astype(int) / 1000

# Insert "Hora" column
fHistory['Hora'] = fHistory['timestamp'].apply(lambda x: x.hour)

# Filter rows where 'URI' is not empty
fHistory = fHistory[fHistory['URI'] != '']

# Insert "Dia da Semana" and "Período" columns
fHistory['Dia da Semana'] = fHistory['data'].apply(lambda x: x.weekday())
fHistory['Período'] = fHistory['Hora'].apply(lambda x: 'Noite' if x >= 18 else 'Tarde' if x >= 17 else 'Manhã' if x >= 9 else 'Madrugada')

# Rename columns
fHistory.rename(columns={
    'data': 'Dia',
    'timestamp': 'End Time',
    'ms_played': 'Seconds Played',
    'Name': 'Música',
    'master_metadata_album_artist_name': 'Artista'
}, inplace=True)

# Transform 'Artista' column to text type
fHistory['Artista'] = fHistory['Artista'].astype(str)

# Replace empty or null 'Album' with 'Artista' if 'Album' is blank
fHistory['Album'] = fHistory.apply(lambda row: row['Artista'] if row['Album'] in [None, '', np.nan] else row['Album'], axis=1)

# Insert "id" column after the delimiter ":" in 'URI'
fHistory['id'] = fHistory['URI'].apply(lambda x: x.split(':')[2] if len(x.split(':')) > 2 else '')

# Final DataFrame
fHistory = fHistory[['Dia', 'End Time', 'Seconds Played', 'Música', 'Artista', 'id', 'Album', 'URI', 'shuffle', 'Hora', 'Dia da Semana', 'Período']]

display(fHistory)

# Save to CSV file
save_file_artist = os.path.join(PATH_OUT, f"fHistory.csv")
fHistory.to_csv(save_file_artist, index=False)



,Dia,End Time,Seconds Played,Música,Artista,id,Album,URI,shuffle,Hora,Dia da Semana,Período
0,2014-09-29,17:53:45,8.135,I Am the Man,Freshlyground,019m7qUQerzKMWNlS7sO2o,Nomvula,spotify:track:019m7qUQerzKMWNlS7sO2o,False,17,0,Tarde
1,2014-09-29,17:59:02,317.106,Vanish,Freshlyground,5ruzfgWfhgusrn772s9dC5,Nomvula,spotify:track:5ruzfgWfhgusrn772s9dC5,False,17,0,Tarde
2,2014-09-29,18:54:09,28.932,Zithande,Freshlyground,6POnCFfPhnIAL5VSh4YqyC,Nomvula,spotify:track:6POnCFfPhnIAL5VSh4YqyC,False,18,0,Noite
3,2014-10-07,00:46:31,324.000,My Little Brown Book,Duke Ellington,4TWObpR5vN8WRzs2FdRRyo,Duke Ellington & John Coltrane,spotify:track:4TWObpR5vN8WRzs2FdRRyo,False,0,1,Madrugada
4,2014-10-07,00:52:22,350.426,Circle,Miles Davis,5Vk50D8fYV6vFGznPKSi6K,The Complete Columbia Studio Recordings Of The...,spotify:track:5Vk50D8fYV6vFGznPKSi6K,False,0,1,Madrugada
...,...,...,...,...,...,...,...,...,...,...,...,...
132593,2023-03-23,10:33:37,0.000,The Power of Awakening | Mindfulness Practices...,None,3S6UXdEFA2hmy686fSfpfp,Bestbookbits,spotify:episode:3S6UXdEFA2hmy686fSfpfp,False,10,3,Manhã
132594,2023-03-23,10:33:37,0.000,Shreya Shankar — Operationalizing Machine Lear...,None,0je6q3rIOTPtDuvMT7rUt6,"Gradient Dissent: Exploring Machine Learning, ...",spotify:episode:0je6q3rIOTPtDuvMT7rUt6,False,10,3,Manhã
132595,2023-03-23,10:33:42,0.000,Central Limit Theoram | Status time | Data Sci...,None,6yhl0RY5ZD0KaOBRRRnFn8,Data Science with Ankit Bansal,spotify:episode:6yhl0RY5ZD0KaOBRRRnFn8,False,10,3,Manhã
132596,2023-03-23,18:32:54,0.000,Central Limit Theoram | Status time | Data Sci...,None,6yhl0RY5ZD0KaOBRRRnFn8,Data Science with Ankit Bansal,spotify:episode:6yhl0RY5ZD0KaOBRRRnFn8,False,18,3,Noite
